In [79]:
from dotenv import dotenv_values
import os
import pandas as pd
import requests
import json
import types
import time 


def get_places(api_key: str, url: str, query: str, page_token:str = None):
    """
    Get places from Google Places API.

    Args:
        api_key: str - Google Places API key
        url: str - Google Places API URL
        query: str - name of the city
    """

    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "nextPageToken,places.id,places.name,places.displayName,places.formattedAddress,places.priceLevel,places.attributions,places.location,places.priceLevel,places.rating,places.userRatingCount,places.servesBreakfast,places.servesBrunch,places.servesCoffee,places.servesDessert,places.servesDinner,places.servesLunch,places.servesVegetarianFood,places.goodForChildren,places.allowsDogs,places.googleMapsLinks",
    }

    data = {"textQuery": query}

    if page_token:
        data["pageToken"] = page_token
  
    try:
        respone = requests.post(url, headers=headers, json=data, timeout=10)
        respone.raise_for_status()
        return respone.json()

    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred: {e}")
        return None

    except Exception as e:
        print(f"Unexpected error occurred: {e}")
        return None





In [73]:
def get_all_city_places(api_key: str, url: str, query: str, max_results: int, page_token: str = None) -> list: 
    """ 
    Get all places for a given query.
    Args:
        api_key: str - Google Places API key
        url: str - Google Places API URL
        query: str - name of the city
        max_results: int - maximum number of results    
    """
    results = []

    while len(results)< max_results:
        print(page_token)
        data = get_places(api_key, url, query, page_token)
        if 'places' in data:
            results.extend(data['places'])
        else:
            break
        
        if data.get('nextPageToken', None):
            page_token = data['nextPageToken']
        else: 
            break
        time.sleep(2)

    return results



In [ ]:
data = get_all_city_places(API_KEY, URL_GOOGLE_APIS_PLACE, query="restaurants in Poznań", max_results=60)

60

In [74]:
def get_all(api_key: str, url: str, query: list,  max_results: int = 40) -> list:
    """
    Get places from all the cities in the list. 
    Args:
        api_key: str - Google Places API key
        url: str - Google Places API URL 
        query: list - list of cities
    
    """
    results = []
    for city in query:
        data = get_all_city_places(api_key, url, city, max_results)
        results.extend(data)
    
    return results
        



In [ ]:

city_restaurants_list = ['restaurants in Poznań', 'restaurants in Kraków']
all_cities_restaurants = get_all(API_KEY, URL_GOOGLE_APIS_PLACE, query=city_restaurants_list, max_results=40)

In [75]:
import json 
path = 'c:\\Users\\hiker\\OneDrive\\Dokumenty\\Arek\\Data_Engineer_Project\\Savory_Spots\\data\\savory_spots.json'
# os.getcwd()

with open(path, 'r') as f:
    test = json.load(f)

In [76]:
len(test)


1920

In [ ]:
import json
import pandas as pd

path = 'c:\\Users\\hiker\\OneDrive\\Dokumenty\\Arek\\Data_Engineer_Project\\Savory_Spots\\src\\config\\list_of_cities.json'
with open(path, 'r', encoding='utf-8') as f:
    test = json.load(f)

test = pd.DataFrame(test)
test =test.explode('City').reset_index(drop=True)
# test['Rank'] = test.groupby('District')['City'].rank(method= 'first', ascending=True).astype(int)
test['Rank'] = test.groupby('District').cumcount() + 1




,District,City,Rank
0,Greater Poland,Poznań,1
1,Greater Poland,Kalisz,2
2,Greater Poland,Konin,3
3,Kuyavia-Pomerania,Bydgoszcz,1
4,Kuyavia-Pomerania,Toruń,2
5,Kuyavia-Pomerania,Włocławek,3
6,Lesser Poland,Kraków,1
7,Lesser Poland,Tarnów,2
8,Lesser Poland,Nowy Sącz,3
9,Łódź,Łódź,1


In [ ]:
path = 'c:\\Users\\hiker\\OneDrive\\Dokumenty\\Arek\\Data_Engineer_Project\\Savory_Spots\\src\\config\\list_of_cities.json'
with open(path, 'r', encoding='utf-8') as f:
    cities_list = json.load(f)

def get_cities_list(config_file_path: str, type_of_spot: str) -> list: 
    """
    Prepare a list of cities for a given type of spot.
    Args:
        config_file_path: str - path to the configuration file
        type_of_spot: str - type of spot
    """
    try:
        with open(config_file_path, 'r', encoding='utf-8') as f:
            cities_list = json.load(f)
    except Exception as e:
        return []
    
    cities_list = cities_list['City']
    cities_list =[city for sublist in cities_list for city in sublist]
    cities_list = [ 'restaurants in ' + city for city in cities_list]

    return cities_list




In [72]:
path = 'c:\\Users\\hiker\\OneDrive\\Dokumenty\\Arek\\Data_Engineer_Project\\Savory_Spots\\src\\config\\list_of_cities.json'

get_cities_list(config_file_path=path, type_of_spot='restaurants in ')[0:3]

['restaurants in Poznań', 'restaurants in Kalisz', 'restaurants in Konin']

In [74]:
test = [1,2,3,]
if test: 
    print('Jest ok ')

Jest ok 
